In [1]:
!pip install tensorflow-model-optimization

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

2025-12-15 18:22:37.407074: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 18:22:37.408540: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 18:22:37.435172: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 18:22:37.435698: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-15 18:22:38.556527: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [3]:
base_path = "/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition"
base_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition'

In [4]:
uci_har_dataset_path = f'{base_path}/data/UCI_HAR'
uci_har_dataset_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/data/UCI_HAR'

In [5]:
if base_path not in sys.path:
    sys.path.append(base_path) 

In [6]:
import src.load_uci_har_dataset as uci_har

In [7]:
Xtrain, ytrain, xtest, ytest, labels = uci_har.load_uci_har_dir(uci_har_dataset_path)

In [8]:
Xtrain.shape, ytrain.shape, xtest.shape, ytest.shape, labels,len(labels)

((7352, 128, 6),
 (7352,),
 (2947, 128, 6),
 (2947,),
 ['WALKING',
  'WALKING_UPSTAIRS',
  'WALKING_DOWNSTAIRS',
  'SITTING',
  'STANDING',
  'LAYING'],
 6)

In [9]:
import src.get_models as models

In [10]:
input_shape = Xtrain.shape[1:]
num_classes = len(labels)
input_shape, num_classes

((128, 6), 6)

In [11]:
import importlib
importlib.reload(models)

<module 'src.get_models' from '/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/src/get_models.py'>

In [12]:
base_model = keras.models.load_model(f'{base_path}/models/gru_model.keras')

2025-12-15 18:22:52.079378: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 18:22:52.109579: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-12-15 18:22:52.229698: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

In [13]:
base_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 6)]          0         
                                                                 
 gru_9 (GRU)                 (None, 128, 64)           13824     
                                                                 
 gru_10 (GRU)                (None, 32)                9408      
                                                                 
 dense_6 (Dense)             (None, 128)               4224      
                                                                 
 batch_normalization_3 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                           

In [14]:
baseline_model_loss, baseline_model_accuracy = base_model.evaluate(
    xtest, ytest)

print('Baseline test accuracy:', baseline_model_accuracy)


2025-12-15 18:22:59.868589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:22:59.869594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:22:59.870139: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

93/93 [==============================] - 2s 19ms/step - loss: 0.3158 - accuracy: 0.9165
Baseline test accuracy: 0.9165253043174744


In [15]:
q_aware_model = models.get_qat_model_dense_only(input_shape, num_classes)

2025-12-15 18:23:06.245244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:23:06.246072: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:23:06.246900: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [18]:
q_aware_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [19]:
q_aware_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 6)]          0         
                                                                 
 conv1d (Conv1D)             (None, 128, 64)           1984      
                                                                 
 batch_normalization (BatchN  (None, 128, 64)          256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128, 64)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 128, 128)          24704     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128)         512       
 hNormalization)                                             

In [20]:
lr_decay_callback = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                      patience=8, min_lr=1e-6, verbose=1)
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
q_aware_model.fit(Xtrain, ytrain, epochs=150, batch_size=64,
                          validation_split=0.1, verbose=1, callbacks=[early_stop, lr_decay_callback])

Epoch 1/150


2025-12-15 18:30:37.193258: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:30:37.194197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:30:37.194882: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

104/104 [==============================] - ETA: 0s - loss: 0.8936 - accuracy: 0.5407

2025-12-15 18:30:48.800770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:30:48.801793: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:30:48.802408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

104/104 [==============================] - 13s 102ms/step - loss: 0.8936 - accuracy: 0.5407 - val_loss: 1.9521 - val_accuracy: 0.1957 - lr: 0.0010
Epoch 2/150
104/104 [==============================] - 10s 95ms/step - loss: 0.6773 - accuracy: 0.6312 - val_loss: 2.0338 - val_accuracy: 0.1889 - lr: 0.0010
Epoch 3/150
104/104 [==============================] - 10s 100ms/step - loss: 0.6258 - accuracy: 0.6625 - val_loss: 0.8372 - val_accuracy: 0.5720 - lr: 0.0010
Epoch 4/150
104/104 [==============================] - 10s 95ms/step - loss: 0.5909 - accuracy: 0.6898 - val_loss: 1.2361 - val_accuracy: 0.6345 - lr: 0.0010
Epoch 5/150
104/104 [==============================] - 10s 95ms/step - loss: 0.5910 - accuracy: 0.6953 - val_loss: 0.7437 - val_accuracy: 0.6481 - lr: 0.0010
Epoch 6/150
104/104 [==============================] - 10s 95ms/step - loss: 0.5888 - accuracy: 0.6988 - val_loss: 0.6141 - val_accuracy: 0.6671 - lr: 0.0010
Epoch 7/150
104/104 [==============================] - 10s 98m

104/104 [==============================] - 11s 107ms/step - loss: 0.1918 - accuracy: 0.9229 - val_loss: 0.8491 - val_accuracy: 0.8247 - lr: 2.5000e-04
Epoch 50/150
104/104 [==============================] - 12s 111ms/step - loss: 0.1894 - accuracy: 0.9232 - val_loss: 0.2398 - val_accuracy: 0.8872 - lr: 1.2500e-04
Epoch 51/150
104/104 [==============================] - 11s 108ms/step - loss: 0.1768 - accuracy: 0.9282 - val_loss: 0.1835 - val_accuracy: 0.9266 - lr: 1.2500e-04
Epoch 52/150
104/104 [==============================] - 11s 109ms/step - loss: 0.1811 - accuracy: 0.9290 - val_loss: 0.5552 - val_accuracy: 0.8519 - lr: 1.2500e-04
Epoch 53/150
104/104 [==============================] - 11s 110ms/step - loss: 0.1672 - accuracy: 0.9356 - val_loss: 0.1778 - val_accuracy: 0.9022 - lr: 1.2500e-04
Epoch 54/150
104/104 [==============================] - 11s 106ms/step - loss: 0.1631 - accuracy: 0.9341 - val_loss: 0.2524 - val_accuracy: 0.8587 - lr: 1.2500e-04
Epoch 55/150
104/104 [=======

Epoch 97/150
104/104 [==============================] - 10s 97ms/step - loss: 0.1309 - accuracy: 0.9498 - val_loss: 0.1241 - val_accuracy: 0.9606 - lr: 1.5625e-05
Epoch 98/150
104/104 [==============================] - 10s 98ms/step - loss: 0.1304 - accuracy: 0.9510 - val_loss: 0.1141 - val_accuracy: 0.9565 - lr: 1.5625e-05
Epoch 99/150
104/104 [==============================] - 10s 97ms/step - loss: 0.1375 - accuracy: 0.9445 - val_loss: 0.0919 - val_accuracy: 0.9701 - lr: 1.5625e-05
Epoch 100/150
104/104 [==============================] - 10s 98ms/step - loss: 0.1271 - accuracy: 0.9509 - val_loss: 0.0979 - val_accuracy: 0.9674 - lr: 1.5625e-05
Epoch 101/150
104/104 [==============================] - 11s 109ms/step - loss: 0.1361 - accuracy: 0.9454 - val_loss: 0.1191 - val_accuracy: 0.9524 - lr: 1.5625e-05
Epoch 102/150
104/104 [==============================] - 11s 110ms/step - loss: 0.1302 - accuracy: 0.9495 - val_loss: 0.1233 - val_accuracy: 0.9443 - lr: 1.5625e-05
Epoch 103/150
104

In [21]:
q_aware_model_loss, q_aware_model_accuracy = q_aware_model.evaluate(
   xtest, ytest)

print('Quant test accuracy:', q_aware_model_accuracy)

93/93 [==============================] - 3s 28ms/step - loss: 0.2562 - accuracy: 0.9270
Quant test accuracy: 0.9270444512367249


In [22]:
import tensorflow as tf
run_model = tf.function(lambda x: q_aware_model(x))
BATCH_SIZE = 1
STEPS = 128
INPUT_SIZE = 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], q_aware_model.inputs[0].dtype))

# model directory.
MODEL_DIR = f"{base_path}/GRU_QAT"
q_aware_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

2025-12-15 18:50:00.325641: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:50:00.326560: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:50:00.327158: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:50:01.592480: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:50:01.593425: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:50:01.594150: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:50:02.635854: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:50:02.637260: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:50:02.637845: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:50:03.531300: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:50:03.532491: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:50:03.533315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_QAT/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_QAT/assets


In [23]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

2025-12-15 18:50:04.800191: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:50:04.800346: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:50:04.800441: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:50:05.403028: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:50:05.403154: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:50:05.403232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:50:05.870665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:50:05.870790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:50:05.870876: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [24]:
with open(f"{base_path}/models/gru_qat.tflite", "wb") as file:
    file.write(quantized_tflite_model)

In [25]:
def evaluate_model(interpreter):
    in_det  = interpreter.get_input_details()[0]
    out_det = interpreter.get_output_details()[0]
    in_idx  = in_det["index"]
    out_idx = out_det["index"]
    in_dtype = in_det["dtype"]     
    in_scale, in_zero = in_det["quantization"]
    print("dtype", in_dtype)

    prediction_digits = []
    for i, test_image in enumerate(xtest):
        if i % 1000 == 0:
            print(f"Evaluated on {i} results so far.")

        xi = np.expand_dims(test_image, axis=0).astype(np.float32)

        if in_dtype == np.int8:
            xi_q = np.clip(np.round(xi / in_scale + in_zero), -128, 127).astype(np.int8)
            interpreter.set_tensor(in_idx, xi_q)
        elif in_dtype == np.uint8:
            xi_q = np.round(xi / in_scale + in_zero).astype(np.uint8)
            interpreter.set_tensor(in_idx, xi_q)
        else:
            interpreter.set_tensor(in_idx, xi)

        interpreter.invoke()

        yo = interpreter.get_tensor(out_det["index"])[0]
        prediction_digits.append(int(np.argmax(yo)))

    print('\n')
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == ytest).mean()
    return accuracy

In [26]:
interpreter = tf.lite.Interpreter(model_path=f"{base_path}/models/gru_qat.tflite")
interpreter.allocate_tensors()

test_accuracy_qat_tflite = evaluate_model(interpreter)
print("quant aware training model accuracy", test_accuracy_qat_tflite)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


dtype <class 'numpy.float32'>
Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.


quant aware training model accuracy 0.8534102477095351


In [27]:
print("GRU QAT model size in MB:", os.path.getsize(f"{base_path}/models/gru_qat.tflite") / float(2**20))

GRU QAT model size in MB: 0.837127685546875
